In [17]:
from tensorflow.python.tools import inspect_checkpoint as chkp
model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
chkp.print_tensors_in_checkpoint_file(model_path+'model.ckpt', tensor_name='', all_tensors=True)

tensor_name:  b1
[ 0.93859965 -0.2740683   0.5565158   1.8145977   0.39980325  1.7180223
  1.3877919  -0.6823048  -0.03705709 -1.267076    1.3749678  -0.84814894
  0.7199019   1.6043197  -0.9926632   1.4425429  -0.04913814 -0.60444176
 -1.4658084   1.1215305  -0.70981634 -1.0913297  -0.4791125   0.5186312
 -0.92166644  1.3596711   0.12717035 -0.07758104 -0.7245661   0.54443514
 -0.03902167 -0.8071262   0.03552718  0.21813162  0.41276887 -0.58524114
 -0.39795962  0.43605673 -0.10470647 -1.0176708   0.7975049  -0.9695002
  0.80834526  0.5748044   0.46487954 -0.35111007 -0.69499594  0.28024042
  0.9043978   0.70213354 -0.27416822  0.45092145  0.5313214   1.0102931
 -0.794642    0.41605055  0.06532101 -0.21521537 -0.6774521   0.7744807 ]
tensor_name:  b2
[-6.2578636e-01  7.3656857e-01  1.4439723e+00  1.0653658e+00
 -3.4646124e-02 -2.6236946e-02 -9.0519691e-01 -8.6343676e-01
  1.6965063e+00 -1.2121990e+00  1.5296267e-03 -2.0002231e-01
  2.0461395e-01 -6.2216133e-01  9.0052330e-01 -6.5566188

In [19]:
# Testing model on fresh data
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops

# Reading the dataset
# Get the dataset "sonar.csv" at 
# https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data
# Just save the file as sonar.csv
def read_dataset(file_name):
    dir_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/"
    df = pd.read_csv(dir_path+file_name,delimiter=',')
    print("Nbr columns: ",len(df.columns))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
    # Encode the dependant variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    print("X.shape",X.shape)
    return (X,Y)
 
# Define the encoder function M => 1, R => 0
def one_hot_encode(labels):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = 2
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode

# Read the dataset
X, Y = read_dataset('new_sonar.csv')

n_dim = X.shape[1] 
n_class = 2

# # Inputs and outputs
# x = tf.placeholder(tf.float32,[None, n_dim])
# y_ = tf.placeholder(tf.float32,[None, n_class])

# Define the number of hidden layers an the
# number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activations
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with sigmoid activations
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoid activations
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with RELU activations
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    # Output layer with linear activations
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out_b']
    return out_layer
 
tf.reset_default_graph() 

model_path = "/Users/claudecoulombe/git/tensorflow_basic_tutorial/model/"
new_saver = tf.train.import_meta_graph(model_path+'model.ckpt.meta')

with tf.Session() as sess:  
    new_saver.restore(sess, tf.train.latest_checkpoint(model_path))

    # Inputs and outputs
    x = tf.placeholder(tf.float32,[None, n_dim])
    y_ = tf.placeholder(tf.float32,[None, n_class])

    # define the weights and the biases for each layer
    weights = {
        'h1': sess.run("h1:0"),
        'h2': sess.run("h2:0"),
        'h3': sess.run("h3:0"),
        'h4': sess.run("h4:0"),
        'out': sess.run("out:0"),
        }
    biases = {
        'b1': sess.run("b1:0"),
        'b2': sess.run("b2:0"),
        'b3': sess.run("b3:0"),
        'b4': sess.run("b4:0"),
        'out_b': sess.run("out_b:0"),
        }

    # Call your model defined

    print("**************************************************************")
    print("* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *")
    print("**************************************************************")
    for i in range(0,8):
        # Call your model defined
        prediction_run = sess.run(tf.argmax(multilayer_perceptron(x, weights, biases), 1), feed_dict={x:X[i].reshape(1,60)})
        accuracy_run = sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(multilayer_perceptron(x, weights, biases), 1), tf.argmax(y_,1)), tf.float32)), feed_dict={x:X[i].reshape(1,60), y_:Y[i].reshape(1,2)})
        print(i,"Original Class: ", int(sess.run(y_[i][1],feed_dict={y_:Y})), " Predicted Values: ", prediction_run[0] )
        print("Accuracy: ",str(accuracy_run*100)+"%")

Nbr columns:  61
X.shape (8, 60)
INFO:tensorflow:Restoring parameters from /Users/claudecoulombe/git/tensorflow_basic_tutorial/model/model.ckpt
**************************************************************
* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *
**************************************************************
0 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
1 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
2 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
3 Original Class:  1  Predicted Values:  1
Accuracy:  100.0%
4 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
5 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
6 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
7 Original Class:  0  Predicted Values:  0
Accuracy:  100.0%
